In [1]:
#!pip3 install PyMySQL

In [2]:
import pymysql
import pandas as pd

In [3]:
conn = pymysql.connect(user='root', password='', database='sql_final')

In [4]:
cursor = conn.cursor()

cursor.execute("SHOW TABLES")

records = cursor.fetchall()



In [5]:
print(records)

(('accident',), ('cause',), ('condi',), ('date',), ('handle',), ('people',))


In [6]:
df = pd.read_csv('./NPA_TMA1_Need.csv')


In [7]:
df

,事故類型及型態子類別名稱,發生年度,發生月份,發生日期,處理單位名稱警局層,發生地點,天候名稱,道路類別-第1當事者-名稱,當事者順位,當事者區分-類別-大類別名稱-車種,當事者屬-性-別名稱,當事者事故發生時年齡,肇因研判子類別名稱-主要,肇事逃逸類別名稱-是否肇逃
0,撞號誌、標誌桿,2022,1.0,20220101.0,雲林縣警察局,雲林縣虎尾鎮雲145公路路燈290570號,晴,市區道路,1.0,機車,男,17.0,未注意車前狀態,否
1,撞號誌、標誌桿,2022,1.0,20220101.0,雲林縣警察局,雲林縣虎尾鎮雲145公路路燈290570號,晴,市區道路,2.0,NaN,無或物(動物、堆置物),-1.0,未注意車前狀態,否
2,追撞,2022,1.0,20220101.0,國道公路警察局,屏東縣九如鄉國道3號 393公里600.0公尺處南側向外側,晴,國道,1.0,小客車,男,34.0,未注意車前狀態,否
3,追撞,2022,1.0,20220101.0,國道公路警察局,屏東縣九如鄉國道3號 393公里600.0公尺處南側向外側,晴,國道,2.0,小貨車(含客、貨兩用),男,43.0,未注意車前狀態,否
4,追撞,2022,1.0,20220101.0,國道公路警察局,屏東縣九如鄉國道3號 393公里600.0公尺處南側向外側,晴,國道,3.0,NaN,無或物(動物、堆置物),-1.0,未注意車前狀態,否
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4420,穿越道路中,2022,12.0,20221231.0,基隆市警察局,基隆市仁愛區仁一路 / 基隆市仁愛區愛四路,雨,市區道路,2.0,機車,男,23.0,未依標誌、標線、號誌或手勢指揮穿越道路,否
4421,穿越道路中,2022,12.0,20221231.0,基隆市警察局,基隆市仁愛區仁一路 / 基隆市仁愛區愛四路,雨,市區道路,3.0,人,男,70.0,未依標誌、標線、號誌或手勢指揮穿越道路,否
4422,其他,2022,12.0,20221231.0,基隆市警察局,基隆市七堵區基隆市七堵區明德二路工東高幹35B8852CC38,雨,市區道路,1.0,小貨車(含客、貨兩用),男,40.0,開啟車門不當而肇事,否
4423,其他,2022,12.0,20221231.0,基隆市警察局,基隆市七堵區基隆市七堵區明德二路工東高幹35B8852CC38,雨,市區道路,2.0,機車,男,64.0,開啟車門不當而肇事,否


In [8]:
for i, row in df.iterrows() :
    datetime = str(int(row['發生日期']))
    year = int(row['發生年度'])
    month = int(row['發生月份'])
    day = int(datetime[-2:])

    data = (datetime, year, month, day)
    query = "SELECT * FROM date WHERE datetime = %s "
    cursor.execute(query, (datetime))
    result = cursor.fetchone()

    # 如果資料不存在，則插入資料
    if not result:
        query = "INSERT INTO date (datetime, year, month, day) VALUES (%s, %s, %s, %s)"
        print(data)
        cursor.execute(query, data)
        print("insert:",datetime, year, month, day)

# 提交事務
conn.commit()

In [9]:
for i, row in df.iterrows() :
    weather = str(row['天候名稱'])
    road_type = str(row['道路類別-第1當事者-名稱'])
    
    data= (weather,road_type)
    
    query = "SELECT * FROM condi WHERE (weather, road_type) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        print(data)
        query = "INSERT INTO condi (weather, road_type) VALUES (%s, %s)"
        cursor.execute(query, data)
        print("insert:",wether, road_type)

# 提交事務
conn.commit()

In [10]:
for i, row in df.iterrows() :
    cause_type = str(row['肇因研判子類別名稱-主要'])
    hit_and_run = str(row['肇事逃逸類別名稱-是否肇逃'])
    
    if hit_and_run == "否":
        hit_and_run=0
    elif hit_and_run == "是":
        hit_and_run=1
    
    data= (cause_type, hit_and_run)
    
    query = "SELECT * FROM cause WHERE (cause_type, hit_and_run) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        print(data)
        query = "INSERT INTO cause (cause_type, hit_and_run) VALUES (%s, %s)"
        cursor.execute(query, data)
        print("insert:",cause_type, hit_and_run)

# 提交事務
conn.commit()

In [11]:
for i, row in df.iterrows() :
    city = str(row['發生地點'])
    station = str(row['處理單位名稱警局層'])
    city=city[0:3]
    
    data= (city, station)
    
    query = "SELECT * FROM handle WHERE (city, station) = (%s, %s)"
    cursor.execute(query, data)
    result = cursor.fetchone()
    
    if not result:
        print(data)
        query = "INSERT INTO handle (city, station) VALUES (%s, %s)"
        cursor.execute(query, data)
        print("insert:", city, station)

# 提交事務
conn.commit()

In [12]:
cursor.close()

In [13]:
conn.close()